# Spotify Songs Analysis - Data Exploration

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from pathlib import Path
import sys

# Add src to path for custom modules
sys.path.append('../src')
from data_processing import load_raw, normalize_columns, basic_clean, save_clean

# Set up paths
DATA_DIR = Path('../data')
RAW_DATA_PATH = DATA_DIR / 'raw' / 'spotify_songs.csv'

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

## 2. Load Data

In [ ]:
# Try to load data with fallback options
def load_dataset():
    """Load dataset with multiple fallback options"""
    possible_files = [
        'spotify_songs.csv',
        'SpotifyFeatures.csv', 
        'tracks.csv',
        'data.csv'
    ]
    
    for filename in possible_files:
        file_path = DATA_DIR / 'raw' / filename
        if file_path.exists():
            print(f"Loading data from: {file_path}")
            return load_raw(file_path)
    
    # If no local file found, try to download from URL
    try:
        print("No local file found. Attempting to download from URL...")
        url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv"
        df = pd.read_csv(url)
        # Save a local copy
        DATA_DIR.mkdir(parents=True, exist_ok=True)
        (DATA_DIR / 'raw').mkdir(exist_ok=True)
        df.to_csv(DATA_DIR / 'raw' / 'spotify_songs.csv', index=False)
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        # Create sample data for demonstration
        print("Creating sample data for demonstration...")
        return create_sample_data()

def create_sample_data():
    """Create sample data if no dataset is available"""
    np.random.seed(42)
    n_samples = 1000
    
    data = {
        'track_name': [f'Song_{i}' for i in range(n_samples)],
        'artist_name': [f'Artist_{np.random.randint(1, 50)}' for _ in range(n_samples)],
        'playlist_genre': np.random.choice(['Pop', 'Rock', 'Hip-Hop', 'Jazz', 'Classical', 'Electronic'], n_samples),
        'danceability': np.random.uniform(0, 1, n_samples),
        'energy': np.random.uniform(0, 1, n_samples),
        'loudness': np.random.uniform(-20, 0, n_samples),
        'acousticness': np.random.uniform(0, 1, n_samples),
        'instrumentalness': np.random.exponential(0.1, n_samples),
        'liveness': np.random.uniform(0, 1, n_samples),
        'valence': np.random.uniform(0, 1, n_samples),
        'tempo': np.random.uniform(60, 200, n_samples),
        'duration_ms': np.random.randint(120000, 360000, n_samples),
        'popularity': np.random.randint(0, 101, n_samples)
    }
    
    # Cap instrumentalness at 1
    data['instrumentalness'] = np.minimum(data['instrumentalness'], 1)
    
    return pd.DataFrame(data)

# Load the data
df = load_dataset()
print(f"Dataset shape: {df.shape}")

In [ ]:
# Initial inspection
print("First 5 rows:")
display(df.head())

print("\nDataset info:")
df.info()

print("\nColumn names:")
print(df.columns.tolist())

## 3. Data Cleaning and Normalization

In [ ]:
# Normalize column names
df_clean = normalize_columns(df)
print("Column names after normalization:")
print(df_clean.columns.tolist())

In [ ]:
# Check for missing values
print("Missing values:")
missing_data = df_clean.isnull().sum()
missing_percent = (missing_data / len(df_clean)) * 100
missing_info = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})
display(missing_info[missing_info['Missing Count'] > 0])

In [ ]:
# Check for duplicates
duplicates = df_clean.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Basic cleaning
df_clean = basic_clean(df_clean)
print(f"Shape after basic cleaning: {df_clean.shape}")

In [ ]:
# Data validation - check ranges for audio features
def validate_audio_features(df):
    """Validate that audio features are within expected ranges"""
    features_ranges = {
        'danceability': (0, 1),
        'energy': (0, 1),
        'acousticness': (0, 1),
        'instrumentalness': (0, 1),
        'liveness': (0, 1),
        'valence': (0, 1),
        'popularity': (0, 100)
    }
    
    print("Data validation results:")
    for feature, (min_val, max_val) in features_ranges.items():
        if feature in df.columns:
            invalid = ~df[feature].between(min_val, max_val)
            if invalid.any():
                print(f"  {feature}: {invalid.sum()} values outside range [{min_val}, {max_val}]")
            else:
                print(f"  {feature}: All values within expected range")

validate_audio_features(df_clean)

In [ ]:
# Filter unrealistic values
print("Filtering unrealistic values...")
initial_shape = df_clean.shape

# Filter based on duration (reasonable song length)
if 'duration_ms' in df_clean.columns:
    df_clean = df_clean[(df_clean['duration_ms'] >= 30000) & (df_clean['duration_ms'] <= 600000)]

# Filter tempo (reasonable BPM range)
if 'tempo' in df_clean.columns:
    df_clean = df_clean[(df_clean['tempo'] >= 40) & (df_clean['tempo'] <= 240)]

print(f"Rows removed: {initial_shape[0] - df_clean.shape[0]}")
print(f"Final shape: {df_clean.shape}")

## 4. Basic Statistics and Data Understanding

In [ ]:
# Descriptive statistics for numerical features
numerical_features = ['danceability', 'energy', 'loudness', 'acousticness', 
                     'instrumentalness', 'liveness', 'valence', 'tempo', 
                     'duration_ms', 'popularity']

# Select only columns that exist in the dataframe
available_numerical = [col for col in numerical_features if col in df_clean.columns]

print("Descriptive statistics for numerical features:")
display(df_clean[available_numerical].describe())

In [ ]:
# Categorical features analysis
categorical_columns = []
for col in ['playlist_genre', 'genre']:
    if col in df_clean.columns:
        categorical_columns.append(col)

if categorical_columns:
    print("Categorical features value counts:")
    for col in categorical_columns:
        print(f"\n{col}:")
        display(df_clean[col].value_counts().head(10))
else:
    print("No categorical columns found with expected names")

## 5. SQL Analysis with SQLite

In [ ]:
# Create in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Save dataframe to SQL
df_clean.to_sql('spotify_songs', conn, index=False, if_exists='replace')

# Example 1: Average audio features by genre
if 'playlist_genre' in df_clean.columns:
    query1 = """
    SELECT playlist_genre,
           COUNT(*) as song_count,
           AVG(danceability) as avg_danceability,
           AVG(energy) as avg_energy,
           AVG(valence) as avg_valence,
           AVG(popularity) as avg_popularity
    FROM spotify_songs
    GROUP BY playlist_genre
    ORDER BY avg_popularity DESC
    """
    
    genre_stats = pd.read_sql_query(query1, conn)
    print("Average audio features by genre:")
    display(genre_stats)

In [ ]:
# Example 2: Popularity distribution analysis
query2 = """
SELECT 
    CASE 
        WHEN popularity < 40 THEN 'Low (0-39)'
        WHEN popularity BETWEEN 40 AND 60 THEN 'Medium (40-60)'
        ELSE 'High (61-100)'
    END as popularity_category,
    COUNT(*) as count,
    ROUND(AVG(danceability), 3) as avg_danceability,
    ROUND(AVG(energy), 3) as avg_energy,
    ROUND(AVG(acousticness), 3) as avg_acousticness
FROM spotify_songs
GROUP BY popularity_category
ORDER BY 
    CASE popularity_category
        WHEN 'Low (0-39)' THEN 1
        WHEN 'Medium (40-60)' THEN 2
        WHEN 'High (61-100)' THEN 3
    END
"""

popularity_stats = pd.read_sql_query(query2, conn)
print("Popularity category analysis:")
display(popularity_stats)

In [ ]:
# Example 3: High energy songs analysis
query3 = """
SELECT 
    playlist_genre,
    COUNT(*) as total_songs,
    SUM(CASE WHEN energy > 0.8 THEN 1 ELSE 0 END) as high_energy_songs,
    ROUND(SUM(CASE WHEN energy > 0.8 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as high_energy_percentage
FROM spotify_songs
WHERE playlist_genre IS NOT NULL
GROUP BY playlist_genre
ORDER BY high_energy_percentage DESC
"""

energy_analysis = pd.read_sql_query(query3, conn)
print("High energy songs by genre:")
display(energy_analysis)

# Close connection
conn.close()

## 6. Save Cleaned Data

In [ ]:
# Save cleaned dataset
save_clean(df_clean, 'clean_spotify.csv')
print("Cleaned dataset saved successfully!")

# Final dataset info
print(f"\nFinal cleaned dataset shape: {df_clean.shape}")
print(f"Columns in cleaned dataset: {list(df_clean.columns)}")

In [ ]:
# Quick visualization of popularity distribution
plt.figure(figsize=(10, 6))
plt.hist(df_clean['popularity'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.xlabel('Popularity Score')
plt.ylabel('Frequency')
plt.title('Distribution of Song Popularity')
plt.grid(True, alpha=0.3)
plt.show()

print("\nData exploration completed successfully!")
print("Next step: Run 02_feature_analysis.ipynb for detailed feature analysis")